# Exploring Embedding Models in LangChain

## Install OpenAI, HuggingFace and LangChain dependencies

In [ ]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install langchain-huggingface==0.1.2

## Enter Open AI and HuggingFace API Tokens

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [ ]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass('Enter HuggingFace Auth Token Key: ')

## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

## Embedding models

The Embeddings class is a class designed for interfacing with text embedding models. There are lots of embedding model providers (OpenAI, Cohere, Hugging Face, etc) - this class is designed to provide a standard interface for all of them.

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query. The former takes as input multiple texts, while the latter takes a single text. The reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

In [5]:
docs = [
    "cats eat and sleep",
    "dogs eat and bark",
    "cars drive fast",
    "vehicles include trucks and cars"
]

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query. The former, `.embed_documents`, takes as input multiple texts, while the latter, `.embed_query`, takes a single text. The reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

- `.embed_query`  will return a list of floats,
- `.embed_documents` returns a list of lists of floats.

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [5]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [6]:
embeddings = openai_embed_model.embed_documents(docs)

In [7]:
len(embeddings)

4

In [8]:
len(embeddings[0])

1536

In [9]:
print(embeddings[0])

[-0.01413801871240139, -0.004937341902405024, -0.008978158235549927, 0.011596787720918655, 0.020226655527949333, 0.026702281087636948, 0.02379985898733139, 0.03508705273270607, -0.005072788335382938, 0.0032442626543343067, 0.050205446779727936, 0.0012069237418472767, 0.03106236271560192, 0.047006335109472275, 0.0034603318199515343, 0.016743749380111694, 0.008088082075119019, 0.012183722108602524, -0.006998061202466488, 0.03549984470009804, 0.03725419566035271, -0.028998417779803276, 0.03212013468146324, -0.035783637315034866, -0.02972079999744892, 0.054746124893426895, -0.008546019904315472, 0.033539097756147385, 0.012119223363697529, -0.029256410896778107, 0.02481893077492714, -0.0406855046749115, -0.011383943259716034, -0.040866099298000336, 0.011409742757678032, 0.023077478632330894, 0.020226655527949333, -0.03147515282034874, -0.006746518425643444, 0.013286641798913479, -0.009332898072898388, 0.014770101755857468, 0.021839112043380737, -0.009455445222556591, -0.009068455547094345, 

In [10]:
docs

['cats eat and sleep',
 'dogs eat and bark',
 'cars drive fast',
 'vehicles include trucks and cars']

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(embeddings)
sim_matrix

array([[1.        , 0.61263542, 0.20346347, 0.19820668],
       [0.61263542, 1.        , 0.26193663, 0.20028985],
       [0.20346347, 0.26193663, 1.        , 0.39918546],
       [0.19820668, 0.20028985, 0.39918546, 1.        ]])

## Open Source Embedding Models on HuggingFace

`langchain-huggingface` integrates seamlessly with LangChain, providing an efficient and effective way to utilize Hugging Face models within the LangChain ecosystem.

`HuggingFaceEmbeddings`uses `sentence-transformers` embeddings. It computes the embedding locally, using your computer resources and allows you to access open or open source embedding LLMs hosted on HuggingFace.

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

# check out model details here: https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
model_name = "mixedbread-ai/mxbai-embed-large-v1"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
)

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

In [6]:
embeddings = hf_embeddings.embed_documents(docs)

In [7]:
len(embeddings)

4

In [8]:
len(embeddings[0])

1024

In [16]:
docs

['cats eat and sleep',
 'dogs eat and bark',
 'cars drive fast',
 'vehicles include trucks and cars']

In [17]:
sim_matrix = cosine_similarity(embeddings)
sim_matrix

array([[1.        , 0.52516037, 0.34221861, 0.33691799],
       [0.52516037, 1.        , 0.31728606, 0.33082586],
       [0.34221861, 0.31728606, 1.        , 0.72253335],
       [0.33691799, 0.33082586, 0.72253335, 1.        ]])

## Build a small search engine!

### Load Knowledgebase documents

In [18]:
documents = [
    'Quantum mechanics describes the behavior of very small particles.',
    'Photosynthesis is the process by which green plants make food using sunlight.',
    "Shakespeare's plays are a testament to English literature.",
    'Artificial Intelligence aims to create machines that can think and learn.',
    'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
    'Biology is the study of living organisms and their interactions with the environment.',
    'Music therapy can aid in the mental well-being of individuals.',
    'The Milky Way is just one of billions of galaxies in the universe.',
    'Economic theories help understand the distribution of resources in society.',
    'Yoga is an ancient practice that involves physical postures and meditation.'
]

In [19]:
len(documents)

10

### Get document embeddings

In [20]:
document_embeddings = openai_embed_model.embed_documents(documents)

### Let's try to find the most similar document for one query

In [21]:
new_text = 'What is AI?'
new_text

'What is AI?'

In [22]:
query_embedding = openai_embed_model.embed_query(new_text)

In [23]:
cosine_similarities = cosine_similarity([query_embedding], document_embeddings)
cosine_similarities

array([[ 0.10205682,  0.09249925, -0.0052707 ,  0.63139958,  0.02335264,
         0.09318008,  0.10770788,  0.07007732,  0.05822082,  0.06608169]])

In [24]:
import numpy as np

documents[np.argmax(cosine_similarities[0])]

'Artificial Intelligence aims to create machines that can think and learn.'

### Create Search Engine function

In [25]:
def semantic_search_engine(query, embedder_model):
  query_embedding = embedder_model.embed_query(query)
  cos_scores = cosine_similarity([query_embedding], document_embeddings)[0]
  top_result_id = np.argmax(cos_scores)
  return documents[top_result_id]

### Try out the function

In [26]:
new_sentence = 'Tell me about AI'
semantic_search_engine(new_sentence, openai_embed_model)

'Artificial Intelligence aims to create machines that can think and learn.'

In [27]:
new_sentence = 'Do you know about the pyramids?'
semantic_search_engine(new_sentence, openai_embed_model)

'The pyramids of Egypt are historical monuments that have stood for thousands of years.'

In [28]:
new_sentence = 'How do plants survive?'
semantic_search_engine(new_sentence, openai_embed_model)

'Photosynthesis is the process by which green plants make food using sunlight.'